<a href="https://colab.research.google.com/github/Langrzyk/dw_matrix_road_sign/blob/master/Tuning%20hyperparameter%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [29]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/matrix/matrix_three/dw_matrix_road_sign


In [0]:
ls

data/  LICENSE  logs/  README.md


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
#utworzenie słownika labeli
df = pd.read_csv('data/signnames.csv')
labels_dict=df.to_dict()['b']

In [0]:
#stworzenie macierzy
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:

def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  logdir = os.path.join('logs',datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(
      X_train, 
      y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs',5),
      verbose=params_fit.get('verbose',1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks= [tensorboard_callback]
      )
  return model

def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)


In [0]:
#dla wszystkich zdj
X_train_gray = color.rgb2gray( X_train ).reshape(-1, 32, 32, 1)
X_test_gray = color.rgb2gray( X_test ).reshape(-1, 32, 32, 1)

In [0]:
def get_cnn_v5(input_shape,num_classes):
  model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu',input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(1024, activation='relu'),
    Dropout(0.3),
    
    Dense(num_classes, activation='softmax'),
  ])
  return model


In [0]:
def train_and_predict(model):
  model_trained = train_model(model, X_train, y_train)

  return predict(model_trained, X_test, y_test)

In [0]:

train_and_predict( get_cnn_v5(input_shape,num_classes) )

Epoch 1/5
272/272 [==============================] - 4s 15ms/step - loss: 2.7969 - accuracy: 0.2511 - val_loss: 1.3530 - val_accuracy: 0.5673
Epoch 2/5
272/272 [==============================] - 4s 14ms/step - loss: 1.1344 - accuracy: 0.6439 - val_loss: 0.4401 - val_accuracy: 0.8649
Epoch 3/5
272/272 [==============================] - 4s 14ms/step - loss: 0.6030 - accuracy: 0.8063 - val_loss: 0.2482 - val_accuracy: 0.9218
Epoch 4/5
272/272 [==============================] - 4s 14ms/step - loss: 0.4014 - accuracy: 0.8721 - val_loss: 0.1256 - val_accuracy: 0.9651
Epoch 5/5
272/272 [==============================] - 4s 14ms/step - loss: 0.3131 - accuracy: 0.9026 - val_loss: 0.0888 - val_accuracy: 0.9733


0.9358276643990929

In [0]:
def get_model(params):
  model = Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu',input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),

    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),

    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),

    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),
    
    Dense(num_classes, activation='softmax'),
  ])
  return model

hyperoct

In [0]:
def func_obj(params):
  model =  get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
  
  
  model.fit(
      X_train, 
      y_train,
      batch_size = int( params.get('batch_size', 128) ),
      epochs = 5,
      verbose = 0
      )
  
  score = model.evaluate(X_test, y_test)
  accuracy = score[1]
  #print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [42]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),

    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 0.7233 - accuracy: 0.7812

 14/138 [==>...........................]
 - ETA: 0s - loss: 0.1210 - accuracy: 0.9643

 25/138 [====>.........................]
 - ETA: 0s - loss: 0.4615 - accuracy: 0.9212

 35/138 [======>.......................]
 - ETA: 0s - loss: 0.3399 - accuracy: 0.9411

 47/138 [=========>....................]
 - ETA: 0s - loss: 0.3440 - accuracy: 0.9275

 59/138 [===========>..................]
 - ETA: 0s - loss: 0.2930 - accuracy: 0.9364


stochastic gradient descent jastrzebski

Three Factors Influencing Minima in SGD

https://arxiv.org/pdf/1711.04623.pdf